In [1]:
from manager import load_obj
import re
import os
from bs4 import BeautifulSoup
import xml.etree.ElementTree as ET

# Grobid Parsing for evaluation (with Ubuntu)


In [2]:
works = load_obj("germanistik")


In [3]:
goldstandard = ["https://www.hausarbeiten.de/document/111212", "https://www.hausarbeiten.de/document/110999", "https://www.hausarbeiten.de/document/74428", "https://www.hausarbeiten.de/document/110765", "https://www.hausarbeiten.de/document/110758", "https://www.hausarbeiten.de/document/110759", "https://www.hausarbeiten.de/document/110760", "https://www.hausarbeiten.de/document/110747", "https://www.hausarbeiten.de/document/110593", "https://www.hausarbeiten.de/document/111174"]

In [4]:
for work in works:
    if work.link in goldstandard:
        text = work.references
        with open(f'goldstandards/as_text/{work.link.split("/")[-1]}.txt', 'w') as f:
            f.write(text)

In [5]:
import sys
sys.path.insert(0,"grobid-0.7.1/grobid_client_python")
from grobid_client.grobid_client import GrobidClient

if __name__ == "__main__":
    client = GrobidClient(config_path="./grobid-0.7.1/grobid_client_python/config.json")


GROBID server is up and running


In [6]:
client.process("processCitationList", "./goldstandards/as_text", output="./goldstandards/grobid/", n=5, verbose = True, include_raw_citations=True, force=True, tei_coordinates= True)

110765.txt
74428.txt
110999.txt
110593.txt
110759.txt
110747.txt
110758.txt
111174.txt
111212.txt
110760.txt
10 files to process in current batch


In [2]:
grobid =[]
#path = os.path.join("/home", "tim", "Desktop", "bachelorarbeit", "goldstandards", "grobid")
path = os.path.join("C:\\", "Users", "Tim", "Desktop", "bachelorarbeit", "goldstandards","grobid")
for filename in os.listdir(path):
    with open(os.path.join(path, filename), 'r', encoding="utf8") as f: # open in readonly mode
        data = f.read()
        data = BeautifulSoup(data, 'xml') 
        grobid.append([data, os.path.basename(f.name)])

In [69]:
grobid[7][0]

<?xml version="1.0" encoding="utf-8"?>
<TEI xmlns="http://www.tei-c.org/ns/1.0" xmlns:mml="http://www.w3.org/1998/Math/MathML" xmlns:xlink="http://www.w3.org/1999/xlink">
<teiHeader/>
<text>
<front/>
<body/>
<back>
<div>
<listBibl>
<biblStruct xml:id="b0">
<monogr>
<author>
<persName><forename type="first">Farhad</forename><surname>Daftary</surname></persName>
</author>
<title level="m">The Ismāˁīlīs. Their History and Doctrines</title>
<meeting><address><addrLine>Cambridge</addrLine></address></meeting>
<imprint>
<date type="published" when="1990">1990</date>
</imprint>
</monogr>
<note type="raw_reference">Farhad Daftary, The Ismāˁīlīs. Their History and Doctrines, Cambridge 1990.</note>
</biblStruct>
<biblStruct xml:id="b1">
<monogr>
<title level="m" type="main">Kurze Geschichte der Ismailiten. Traditionen einer muslimischen Glaubensgemeinschaft</title>
<author>
<persName><forename type="first">Farhad</forename><surname>Daftary</surname></persName>
</author>
<imprint>
<date type="pub

In [3]:
def find_author(soup):
    authors = soup.find_all("author")
    raw_reference = soup.find("note", {"type" : "raw_reference"}).text.lower()
    aut = ""
    for name in authors:
        surname = name.find("surname")
        forename = name.find("forename", {"type" : "first"})
        middlename= name.find("forename", {"type" : "middle"})
        genname = name.find("genName")
        if surname and forename:
            if raw_reference.find(forename.text.lower()) < raw_reference.find(surname.text.lower()):
                if aut != "":
                    aut += forename.text
                else:
                    aut = forename.text
              
                aut+= middlename.text if middlename else ""
                aut+= surname.text
            else:
                if aut != "":
                    aut += surname.text if surname else ""
                else:
                    aut = surname.text if surname else ""
            
                aut += forename.text if forename else ""
                aut += middlename.text if middlename else ""
                aut += genname.text if genname else ""
        else:
            
            if aut != "":
                aut += surname.text if surname else ""
            else:
                aut = surname.text if surname else ""
            
            aut += forename.text if forename else ""
            aut += middlename.text if middlename else ""
            aut += genname.text if genname else ""
    
    return aut


In [4]:
def find_title(soup):
    title = soup.find("analytic")
    if title != None:
        if title.find("title", {"level" : "a"}) != None:
            return title.find("title", {"level" : "a"}).text
        else: 
            return ""
    else:
        if soup.find("monogr").find("title", {"level" : "m"}):
            return soup.find("monogr").find("title", {"level" : "m"}).text
        else: 
            return ""

In [5]:
def find_journal(soup):
    journal = soup.find("title", {"level" : "j"})
    if journal:
        return journal.text
    else:
        return ""

In [6]:
def find_container_title(soup):
    check = soup.find("analytic")
    if check:
        if soup.find("monogr").find("title", {"level": "m"}):
            return soup.find("monogr").find("title", {"level": "m"}).text
        else:
            return ""
    else: 
        return ""

In [7]:
def find_volume(soup):
    volume = soup.find("biblScope", {"unit" : "volume"})
    if volume:
        return volume.text;
    else:
        return ""

In [8]:
def find_editor(soup):
    editor = soup.find_all("editor")
    raw_reference = soup.find("note", {"type" : "raw_reference"}).text.lower()
    aut = ""
    for ed in editor:
        if ed.find("persName"):
            surname = ed.find("surname")
            forename = ed.find("forename", {"type" : "first"})
            middlename= ed.find("forename", {"type" : "middle"})
            genname = ed.find("genName")
            if surname and forename:
                if raw_reference.find(forename.text.lower()) < raw_reference.find(surname.text.lower()):
                    if aut != "":
                        aut += forename.text
                    else:
                        aut = forename.text

                    aut+= middlename.text if middlename else ""
                    aut+= surname.text
                else:
                    if aut != "":
                        aut += surname.text if surname else ""
                    else:
                        aut = surname.text if surname else ""

                    aut += forename.text if forename else ""
                    aut += middlename.text if middlename else ""
                    aut += genname.text if genname else ""
            else:

                if aut != "":
                    aut += surname.text if surname else ""
                else:
                    aut = surname.text if surname else ""

                aut += forename.text if forename else ""
                aut += middlename.text if middlename else ""
                aut += genname.text if genname else ""

        else:
            aut = ed.text

    return aut

In [9]:
def find_publisher(soup):
    publisher = soup.find("publisher")
    if publisher:
        return publisher.text
    else:
        return ""

In [10]:
def find_location(soup):
    location = soup.find("addrLine")
    if location:
        return location.text
    else:
        return ""

In [11]:
def find_date(soup):
    date = soup.find("date")
    if date:
        return date.text
    else:
        return ""

In [12]:
def find_pages(soup):
    pages = soup.find("biblScope", {"unit" : "page"})
    if pages:
        if pages.has_attr("from") and pages.has_attr("to"):
            return str(pages["from"]) + "-" + str(pages["to"])
        else:
            return ""
    else:
        return ""

In [13]:
def find_url(soup):
    url = soup.find("ptr")
    if url:
        return url["target"]
    else:
        return ""

In [14]:
count = 0
for entrys in grobid:
    for refs in entrys[0]:
        data = refs.find_all("biblStruct")
        name = entrys[1].split(".")[0]
        tree = ET.ElementTree("tree")
        document = ET.Element("dataset")
        for ref in data:
            node1 = ET.SubElement(document, "sequence")
            author = find_author(ref)
            title = find_title(ref)
            journal = find_journal(ref)
            container_title = find_container_title(ref)
            volume = find_volume(ref)
            editor = find_editor(ref)
            publisher = find_publisher(ref)
            location = find_location(ref)
            date = find_date(ref)
            pages = find_pages(ref)
            url = find_url(ref)
            
            node2 = ET.SubElement(node1, "author")
            node2.text = author
            node3 = ET.SubElement(node1, "title")
            node3.text = title
            node4 = ET.SubElement(node1, "journal")
            node4.text = journal
            node5 = ET.SubElement(node1, "container-title")
            node5.text = container_title
            node6 = ET.SubElement(node1, "volume")
            node6.text = volume
            node7 = ET.SubElement(node1, "editor")
            node7.text = editor
            node8 = ET.SubElement(node1, "publisher")
            node8.text = publisher
            node9 = ET.SubElement(node1, "location")
            node9.text = location
            node10 = ET.SubElement(node1, "date")
            node10.text = date
            node11 = ET.SubElement(node1, "pages")
            node11.text = pages
            node12 = ET.SubElement(node1, "url")
            node12.text = url
        tree._setroot(document)
        tree.write(f"goldstandards/grobid_xml/{name}.xml", encoding = "UTF-8", xml_declaration = True)  
        print(name)

105327
107339
107391
107833
108636
110593
110747
110758
110759
110760
110765
110999
111174
111212
74428


In [67]:
for entrys in grobid:
    for refs in entrys[0]:
        data = refs.find_all("biblStruct")
        name = entrys[1].split(".")[0]
        print(name)
        for ref in data:

            author = find_author(ref)
            print(author)
            title = find_title(ref)
            journal = find_journal(ref)
            container_title = find_container_title(ref)
            volume = find_volume(ref)
            editor = find_editor(ref)
            publisher = find_publisher(ref)
            location = find_location(ref)
            date = find_date(ref)
            pages = find_pages(ref)
            url = find_url(ref)
        print("##########################################################################")

110593
BarthesRoland
BeierAntje
BlüherKarlAlfred
DauerBernd
EageltonTerry
EsslinMartin
HeiddegerMartin
IserWolfgang
LacanJaques: DasSpiegelstadium
MiesGerhard
Robbe-Grillet
Robbe-Grillet
Robbe-Grillet
Robbe-Grillet
StigleggeMarcus
ZhonshangLi
##########################################################################
110747
BartensA
BolléeA
BuscheC
GlückH
HolmJA
JesurunBA
MaurerPTStolzVaria Creolica
MaurerP
MunteanuD
WeißJ


HarrisF
WunderlichD



##########################################################################
110758
Quellen
AbūZaid As-Sīrāfī
AlīAkbar-E Ḫeṭā’ī, Ḫeṭāynāme.VĪrağ Afšār
IbnBaṭṭūṭa
Al-Masˤūdī
WBauer
CEBosworthMHartmannStichwort „aṣ-Ṣīn
HelmutBrinker
JCBürgel
EConze
ACotterellYYap
WEichhorn
HFrankeRolfTrauzettel
JGernet
HHärtelIndischePlastik
PHeine
PKahle
A
KMyliusDerBuddhismusSeine Lehre Und Seine Geschichte
TNagelDerKoran
SAḥmad
FCReiter
IRobinet
HSchmidt-Glintzer
HWSchumann
RAStein
RTaylor; DTwitchettFWMote
GFranz-Willing
AZeki Velidi ToğanStichwort„ˤalīAkbarKh